<a href="https://colab.research.google.com/github/butchland/fastai_nb_explorations/blob/master/fastai_scratch_with_tpu_mnist_4_experiment4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
assert os.environ['COLAB_TPU_ADDR'], 'Make sure to select TPU from Edit > Notebook settings > Hardware accelerator'

In [2]:
!curl https://course.fast.ai/setup/colab | bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   322  100   322    0     0   1511      0 --:--:-- --:--:-- --:--:--  1511
Updating fastai...
Done.


In [3]:
VERSION = "20200325"  #@param ["1.5" , "20200325", "nightly"]
!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
!python pytorch-xla-env-setup.py --version $VERSION

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  4139  100  4139    0     0  71362      0 --:--:-- --:--:-- --:--:-- 71362
Updating TPU and VM. This may take around 2 minutes.
Updating TPU runtime to pytorch-dev20200325 ...
     |████████████████████████████████| 61kB 2.6MB/s 
Uninstalling torch-1.5.1+cu101:
  Found existing installation: google-api-python-client 1.7.12
    Uninstalling google-api-python-client-1.7.12:
      Successfully uninstalled google-api-python-client-1.7.12
Done updating TPU runtime
  Successfully uninstalled torch-1.5.1+cu101
Uninstalling torchvision-0.6.1+cu101:
  Successfully uninstalled torchvision-0.6.1+cu101
Copying gs://tpu-pytorch/wheels/torch-nightly+20200325-cp36-cp36m-linux_x86_64.whl...
\ [1 files][ 83.4 MiB/ 83.4 MiB]                                                
Operation completed over 1 objects/83.4 MiB.                             

In [2]:
!pip freeze | grep torchvision
!pip freeze | grep torch-xla

torchvision==0.6.0a0+3c254fb
torch-xla==1.6+e788e5b


In [6]:
!pip install fastcore --upgrade

In [7]:
!pip install fastai2 --upgrade

     |████████████████████████████████| 194kB 3.3MB/s 


In [8]:
pip install fastai --upgrade

Requirement already up-to-date: fastai in /usr/local/lib/python3.6/dist-packages (1.0.61)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [3]:
%cd /content/drive/My\ Drive/course-v4/
!pwd

/content/drive/My Drive/course-v4
/content/drive/My Drive/course-v4


In [10]:
!pip install -r requirements.txt

     |████████████████████████████████| 51kB 1.9MB/s 
     |████████████████████████████████| 51kB 4.9MB/s 
     |████████████████████████████████| 1.1MB 10.8MB/s 
     |████████████████████████████████| 40kB 2.9MB/s 
     |████████████████████████████████| 92kB 8.4MB/s 
     |████████████████████████████████| 61kB 6.9MB/s 
     |████████████████████████████████| 51kB 5.1MB/s 
     |████████████████████████████████| 2.7MB 36.5MB/s 


In [4]:
%cd nbs

/content/drive/My Drive/course-v4/nbs


In [12]:
!pwd

/content/drive/My Drive/course-v4/nbs


### Start of import libraries

In [5]:
from fastai2.vision.all import *

In [6]:
from utils import *

In [7]:
path = untar_data(URLs.MNIST_SAMPLE)

In [8]:
Path.BASE_PATH = path

In [9]:
path.ls()

(#3) [Path('valid'),Path('labels.csv'),Path('train')]

### Import torch xla libraries

In [10]:
import torch

In [11]:
import torch_xla

In [12]:
import torch_xla.core.xla_model as xm

In [13]:
class WrapperOpt:
    def __init__(self, f):
        self.f = f 
    def __call__(self, *args, **kwargs):
        opt = self.f(*args, **kwargs)
        def mystep():
            xm.optimizer_step(opt,barrier=True)
        opt.step = mystep
        return opt
    
def wrap_xla_optim(opt):
    w = WrapperOpt(opt)
    return w

### Get TPU Device

In [14]:
tpu_dev = xm.xla_device()

In [15]:
tpu_dev

device(type='xla', index=1)

In [16]:
datablock = DataBlock(
    blocks=(ImageBlock(cls=PILImageBW),CategoryBlock),
    get_items=get_image_files,
    splitter=GrandparentSplitter(),
    get_y=parent_label,
    item_tfms=Resize(28),
    batch_tfms=[])


In [17]:
dls = datablock.dataloaders(path,device=tpu_dev)

In [18]:
adam_xla_opt = wrap_xla_optim(Adam)

In [18]:
sgd_xla_opt = wrap_xla_optim(SGD)

In [19]:
learner = cnn_learner(dls, resnet18, metrics=accuracy, 
                      loss_func=F.cross_entropy, opt_func=sgd_xla_opt)

In [39]:
params = learner.model.parameters()

In [40]:
?params

In [42]:
params = L(params)

In [43]:
len(params)

66

In [44]:
type(params[0])

torch.nn.parameter.Parameter

In [45]:
params[0].shape

torch.Size([64, 3, 7, 7])

In [46]:
params[0].device

device(type='xla', index=1)

In [20]:
learner.fine_tune(1)

RecursionError: ignored

In [21]:
simple_net = nn.Sequential(
    nn.Linear(28*28,30),
    nn.ReLU(),
    nn.Linear(30,1)
)

In [22]:
simple_learner = Learner(dls, simple_net, loss_func=F.cross_entropy,opt_func=sgd_xla_opt)

In [23]:
(L(simple_learner.model.parameters()))[0].device

device(type='xla', index=1)

In [ ]:

simple_learner.fit(40,0.1)

epoch,train_loss,valid_loss,time
